In [1]:
import transformers
import pandas as pd
import json
import os

In [2]:
outerFolder = "/shared/3/projects/benlitterer/agendaSetting/testingHtml"

dfList = []
for folderNum in os.listdir(outerFolder): 
    innerFolder = outerFolder + "/" + folderNum 
    for innerFileName in os.listdir(innerFolder): 
        innerPathName = outerFolder + "/" + folderNum + "/" + innerFileName
        if "json" in innerPathName: 
            with open(innerPathName, "r") as f: 
                data = json.loads(f.read())
            df = pd.json_normalize(data)
            df["id"] = innerFileName.split(".")[0]
            dfList.append(df)
        

In [ ]:
fullDf = pd.concat(dfList).reset_index(drop=True)

In [ ]:
leanDf = fullDf[["id", "title", "url", "source_url", "text"]]

In [ ]:
groundTruths = pd.read_csv("/shared/3/projects/benlitterer/agendaSetting/final_evaluation_data.csv")
print(groundTruths.shape)
groundTruths.head(3)

In [ ]:
#get just the english pairs 
groundTruths.loc[(groundTruths["url1_lang"] == "en") & (groundTruths["url2_lang"] == "en")].shape

In [ ]:
def splitIds(inStr): 
    return inStr.split("_")
#groundTruths[["id1", "id2"]] = groundTruths["pair_id"].apply(splitIds)
groundTruths["id1"] = [item[0] for item in groundTruths["pair_id"].apply(splitIds)]
groundTruths["id2"] = [item[1] for item in groundTruths["pair_id"].apply(splitIds)]

In [ ]:
#This used to work but stopped working potentially due to pandas version error? 
#groundTruths[["id1", "id2"]] = groundTruths["pair_id"].str.split("_", expand=True)

In [ ]:
leanDf = leanDf[["id", "text"]]

In [ ]:
#merge text for the first id, id1
leanDf1 = leanDf.rename(columns = {"id":"id1","text":"text1"})
firstMerge = pd.merge(groundTruths, leanDf1, how="left", on="id1")
firstMerge.head(3)

In [ ]:
leanDf2 = leanDf.rename(columns = {"id":"id2","text":"text2"})
merged = pd.merge(firstMerge, leanDf2, how="left", on="id2")
merged.head(3)

In [24]:
#need a more thoughtful way to address na values.
#only drop when head and tail are both nan? 
#mergedFull = merged.dropna(subset=["text1", "text2"])

In [25]:
mergedFull.shape

(4689, 18)

In [27]:
mergedEn = mergedFull[(mergedFull["url1_lang"] == "en") & (mergedFull["url2_lang"] == "en")]
mergedEn.shape

(234, 18)

In [28]:
mergedEn.to_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/enTestData.csv", sep="\t")